# Finding ground states with DMRG

In [1]:
using MPStates, LinearAlgebra, Test

Let's first define an `Mps` and create a Hubbard Hamiltonian like
$$ H = 0.5\sum^{L-1}_{i=1} c^\dagger_i c_{i+1} + 0.7\sum^{L-1}_{i=1} n_i n_{i+1} $$

In [2]:
# Number of sites.
L = 10
# Type of the Mps.
T = Float64
# Physical dimension.
d = 2

psi = Mps(Float64, L, "W");

# Matrices that describe the Hamiltonian.
# Hopping matrix.
J = 0.5*Symmetric(diagm(1 => ones(L-1)))
# Interaction matrix.
V = 0.7*diagm(1 => ones(L-1))

# Build the Hamiltonian.
H = Mpo(T, L, d)
add_ops!(H, "c+", "c", J, ferm_op="Z")
add_ops!(H, "n", "n", V);

# DMRG minimization algorithm

This is done with the `minimize!` function. It accepts as input an `Mps`, the objective `Mpo`, and a set of minimization options. These options are a structure `MinimizeOpts` with fields:
* `algorithm::String`: DMRG algorithm used for minimizing the state. Can be:
  - `"DMRG1"` for one site DMRG.
  - `"DMRG2"` for two site DMRG.
  - `"DMRG3S"` for strictly single site DMRG with subspace expansion, an improved version of the one site DMRG algorithm that doesn't get trapped in local minima.
* `tol::Float64`: stop the algorithm when the change in the variance of the
        state is less than `tol`.
* `max_sweeps::Int`: maximum number of sweeps allowed.
* `debug::Int`: output information at every step of the minmization:
  * `0`: no info given.
  * `1`: energy, variance and their variation with respect to their last
       value after every right + left sweep.
  * `2`: energy and size of the local Hamiltonian at every step of every
            sweep.
* `sweep_dims::Vector{Int}`: maximum bond dimension at every sweep.

There are some short functions to start the options with predefined values.

In [3]:
# Maximum bond dimension.
m = 60
min_opts1 = MinimizeOpts(m, "DMRG1")
min_opts1_with_some_debugging = MinimizeOpts(m, "DMRG1", debug=1)

# Bond dimension of the sweeps that we want.
sweep_dims = [10, 10, 10, 20, 20, 20, 40, 40, 40, 40, 40, 40]
min_opts2 = MinimizeOpts(sweep_dims, "DMRG1")
min_opts2_with_some_debugging = MinimizeOpts(sweep_dims, "DMRG1", debug=1)

# Now let's find the ground state.
E, var = minimize!(psi, H, min_opts2_with_some_debugging)

println("The energy after each sweep was: $E")

Done sweep 2, bond dimension: 10
    E: -2.281251e+00, ΔE: -3.18e+00
    var: 3.373455e-01, Δvar: 2.97e-01
Done sweep 3, bond dimension: 10
    E: -2.334551e+00, ΔE: -5.33e-02
    var: 2.500887e-01, Δvar: -8.73e-02
Done sweep 4, bond dimension: 20
    E: -2.335560e+00, ΔE: -1.01e-03
    var: 2.409670e-01, Δvar: -9.12e-03
Done sweep 5, bond dimension: 20
    E: -2.335593e+00, ΔE: -3.35e-05
    var: 2.393156e-01, Δvar: -1.65e-03
Done sweep 6, bond dimension: 20
    E: -2.335595e+00, ΔE: -1.37e-06
    var: 2.389822e-01, Δvar: -3.33e-04
Done sweep 7, bond dimension: 40
    E: -2.335595e+00, ΔE: -5.86e-08
    var: 2.389135e-01, Δvar: -6.87e-05
Done sweep 8, bond dimension: 40
    E: -2.335595e+00, ΔE: -2.53e-09
    var: 2.388993e-01, Δvar: -1.42e-05
Done sweep 9, bond dimension: 40
    E: -2.335595e+00, ΔE: -1.10e-10
    var: 2.388963e-01, Δvar: -2.95e-06
Done sweep 10, bond dimension: 40
    E: -2.335595e+00, ΔE: -4.78e-12
    var: 2.388957e-01, Δvar: -6.14e-07
Done sweep 11, bond dimensio